In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import keras
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D
from tqdm.notebook import tqdm
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split as tts
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator # option 1
from keras.callbacks import EarlyStopping

In [27]:
def createframe(di):
    imagepaths = []
    labels = []
    for label in os.listdir(di):
        for imagename in os.listdir(os.path.join(di, label)):
            imagepaths.append(os.path.join(di, label, imagename))
            labels.append(label)
    return imagepaths, labels

In [28]:
def extract_features(images, trates):
    features = []
    for image in tqdm(images):
        if trates == 'train':
            img = load_img(image, target_size=(200, 200))
            img = tf.image.random_crop(img, size=[175,175,3])
            img = tf.image.resize(img, (175, 175))
        elif trates == 'test':
            img = load_img(image, target_size=(175, 175))
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(features.shape[0], 175, 175, 3)  # Reshape all images in one go
    return features

In [29]:
imagepaths, labels = createframe('/content/drive/MyDrive/Data/Train')
train = pd.DataFrame()
train['image'] = imagepaths
train['labels'] = labels
x_train, x_test, y_train, y_test = tts(imagepaths, labels, test_size = 0.1, random_state=42) #new
x_trainf = extract_features(x_train, 'train') / 255 #new

  0%|          | 0/720 [00:00<?, ?it/s]

In [19]:
le = LabelEncoder()
le.fit(y_train) #new
y_trainf = le.transform(y_train) #new
y_trainf = keras.utils.to_categorical(y_trainf, num_classes=2) #new
print(y_trainf)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [20]:

# Initialize the model
model = keras.models.Sequential()

# Convolutional Block 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(175, 175, 3)))
model.add(MaxPooling2D((2, 2)))

# Convolutional Block 2
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convolutional Block 3
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convolutional Block 4
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convolutional Block 5 (if input resolution permits)
model.add(Conv2D(1024, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))


# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Binary classification

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x=x_trainf, y=y_trainf, batch_size=25, epochs=80)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 391ms/step - accuracy: 0.5255 - loss: 0.6989
Epoch 2/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 54ms/step - accuracy: 0.5058 - loss: 0.7039
Epoch 3/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.5219 - loss: 0.6924
Epoch 4/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.5968 - loss: 0.6758
Epoch 5/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.7890 - loss: 0.4867
Epoch 6/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.8600 - loss: 0.3623
Epoch 7/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.8309 - loss: 0.4921
Epoch 8/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.8666 - loss: 0.3724
Epoch 9/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.8892 - loss: 0.3185
Epoch 10/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9157 - loss: 0.2456
Epoch 11/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9320 - loss: 0.2426
Epoch 12/80
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accura

In [21]:
x_testf = extract_features(x_test, 'test')
result = model.predict(x_testf)

dresult = le.inverse_transform(np.argmax(result, axis=1))

outframe = pd.DataFrame()
outframe['img'] = x_test
outframe['actual'] = y_test
outframe['pred'] = dresult

  0%|          | 0/81 [00:00<?, ?it/s]

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 962ms/step


In [22]:
testp = testi = []
for imagename in os.listdir('/content/drive/MyDrive/Test_Images'):
    testp.append(os.path.join('/content/drive/MyDrive/Test_Images', imagename))
test = pd.DataFrame()
test['image'] = testp
test_features = extract_features(testp, 'test')
x_test = test_features

result = model.predict(x_test)
dresult = []
for row in result:
    if row[0] < 0.5:
        dresult.append("Real")
    elif row[0] >= 0.5:
        dresult.append("AI")

testi = [x.partition("Test_Images/")[2] for x in testp]

output = pd.DataFrame()
output['Id'] = testi
output['Label'] = dresult
l = []
for i in output['Id']:
    l.append(i[6:-4])
output['no'] = l
output['no'] = output['no'].astype(int)
output = output.sort_values(by='no', ascending = True)
output = output.drop('no', axis=1)
print(output)

  0%|          | 0/200 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step
                Id Label
41     image_0.jpg    AI
19     image_1.jpg  Real
103    image_2.jpg  Real
90     image_3.jpg  Real
104    image_4.jpg  Real
..             ...   ...
147  image_195.jpg  Real
96   image_196.jpg  Real
136  image_197.jpg  Real
108  image_198.jpg  Real
172  image_199.jpg  Real

[200 rows x 2 columns]


In [23]:
newid = []
for i in output['Id']:
    newid.append(i[:-4])
output['Id'] = newid
print(output)

            Id Label
41     image_0    AI
19     image_1  Real
103    image_2  Real
90     image_3  Real
104    image_4  Real
..         ...   ...
147  image_195  Real
96   image_196  Real
136  image_197  Real
108  image_198  Real
172  image_199  Real

[200 rows x 2 columns]


In [17]:
output.to_csv('PredictionNewKeshav.csv', index=False)